In [2]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

from numpy import pi 
from sympy import Function, simplify

# from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16

from sympy.utilities.lambdify import implemented_function

from sympy import lambdify

from sympy.utilities.lambdify import lambdastr

# 1 Viscous Swimmer

## 1. Variable Length Jacobian

### Symbolic variables for our robot configuration

In [3]:
(theta_1,
 theta_2,
 theta_3,
 L_1,
 L_2,
 L_3,
 B) = sp.symbols("""
                    theta_1,
                    theta_2,
                    theta_3,
                    l_1,
                    l_2,
                    l_3,
                    B
                    """,real=True)


# theta_1 = Function('theta_1', real=True)(t)
# theta_2 = Function('theta_2', real=True)(t)
# theta_3 = Function('theta_3', real=True)(t)

def sym_to_np(T):
    return np.array(T).astype(np.float64)

### Generic 2D Transformation Function

In [4]:
def T(theta,L):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), L*sp.cos(theta)],
                      [sp.sin(theta),  sp.cos(theta), L*sp.sin(theta)],
                      [            0,              0,               1]])

### Zero Position

In [5]:
M = sp.Matrix([[0],[0],[1]])

### Formward Kinematics 

In [6]:
# Kinematics of the First Joint
def T1(theta_1,L_1,s):
    return T(theta_1,s*L_1) 

# Kinematics of the Second Joint
def T2(theta_1,theta_2,L_1,L_2,s):
    return T(theta_1,L_1) @ T(theta_2,s*L_2) 

# Kinematics of the Third Joint
def T3(theta_1,theta_2,theta_3,L_1,L_2,L_3,s):
    return T(theta_1,L_1) @ T(theta_2,L_2) @ T(theta_3,s*L_3) 

In [7]:
FK1 = (T1(theta_1,L_1,B) @ M) [:-1,0] 
FK1

Matrix([
[B*l_1*cos(theta_1)],
[B*l_1*sin(theta_1)]])

In [8]:
FK2 = simplify(T2(theta_1,theta_2,L_1,L_2,B) @ M) [:-1,0] 
FK2


Matrix([
[B*l_2*cos(theta_1 + theta_2) + l_1*cos(theta_1)],
[B*l_2*sin(theta_1 + theta_2) + l_1*sin(theta_1)]])

In [9]:
FK3 = simplify(T3(theta_1,theta_2,theta_3,L_1,L_2,L_3,B) @ M) [:-1,0] 
FK3

Matrix([
[B*l_3*cos(theta_1 + theta_2 + theta_3) + l_1*cos(theta_1) + l_2*cos(theta_1 + theta_2)],
[B*l_3*sin(theta_1 + theta_2 + theta_3) + l_1*sin(theta_1) + l_2*sin(theta_1 + theta_2)]])

### Jacobian

In [93]:
# I am extracting the position part of each translation matrix for the jacobian
X1 = sp.Matrix([theta_1])
J1 = FK1.jacobian(X1).row_join(sp.Matrix([[0,0],[0,0]]))
J1

Matrix([
[-B*sin(theta_1), 0, 0],
[ B*cos(theta_1), 0, 0]])

In [94]:
X2 = sp.Matrix([theta_1,theta_2])
J2 = FK2.jacobian(X2).row_join(sp.Matrix([[0],[0]]))
J2

Matrix([
[-B*sin(theta_1 + theta_2) - sin(theta_1), -B*sin(theta_1 + theta_2), 0],
[ B*cos(theta_1 + theta_2) + cos(theta_1),  B*cos(theta_1 + theta_2), 0]])

In [95]:
X3 = sp.Matrix([theta_1,theta_2,theta_3])
J3 = simplify(FK3.jacobian(X3))
J3

Matrix([
[-B*sin(theta_1 + theta_2 + theta_3) - sin(theta_1) - sin(theta_1 + theta_2), -B*sin(theta_1 + theta_2 + theta_3) - sin(theta_1 + theta_2), -B*sin(theta_1 + theta_2 + theta_3)],
[ B*cos(theta_1 + theta_2 + theta_3) + cos(theta_1) + cos(theta_1 + theta_2),  B*cos(theta_1 + theta_2 + theta_3) + cos(theta_1 + theta_2),  B*cos(theta_1 + theta_2 + theta_3)]])

## 2. Joint Motion Function

In [235]:
t = np.linspace(1,2*np.pi,100)
theta1 = pi/2*np.sin(-t)
theta2 = -pi/2*np.cos(-0.5*t - 4.6)**8 
theta3 = -pi/2*np.cos(-0.5*t - 4.6)**8 

dtheta1 = -pi/2*np.cos(-t)
dtheta2 = -pi/2*np.sin(-0.5*t - 4.6)**8 
dtheta3 = -pi/2*np.sin(-0.5*t - 4.6)**8 

## 3. Evaluating Forward Kinematics at s and t

In [172]:
S = np.linspace(0,1,11)

# Subsituting for Link Lengths
LINKS = [1,1,1]
link1, link2, link3 = LINKS

J1 = J1.subs([[L_1,link1]])
J2 = J2.subs([[L_1,link1],[L_2,link2]])
J3 = J3.subs([[L_1,link1],[L_2,link2],[L_3,link3]])

FK1 = FK1.subs([[L_1,link1]])
FK2 = FK2.subs([[L_1,link1],[L_2,link2]])
FK3 = FK3.subs([[L_1,link1],[L_2,link2],[L_3,link3]])

FK1st = np.zeros((len(S),len(t),2,1));
FK2st = np.zeros((len(S),len(t),2,1));
FK3st = np.zeros((len(S),len(t),2,1));

for i in range(len(S)):
    for j in range(len(t)):
        FK1st[i][j] = sym_to_np(FK1.subs([[B,S[i]],[theta_1,theta1[j]]]))
        FK2st[i][j] = sym_to_np(FK2.subs([[B,S[i]],[theta_1,theta1[j]],[theta_2,theta2[j]]]))
        FK3st[i][j] = sym_to_np(FK3.subs([[B,S[i]],[theta_1,theta1[j]],[theta_2,theta2[j]],[theta_3,theta3[j]]]))
        




In [173]:
J1st = np.zeros((len(S),len(t),2,3));
J2st = np.zeros((len(S),len(t),2,3));
J3st = np.zeros((len(S),len(t),2,3));

for i in range(len(S)):
    for j in range(len(t)):
        J1st[i][j] = sym_to_np(J1.subs([[B,S[i]],[theta_1,theta1[j]]]))
        J2st[i][j] = sym_to_np(J2.subs([[B,S[i]],[theta_1,theta1[j]],[theta_2,theta2[j]]]))
        J3st[i][j] = sym_to_np(J3.subs([[B,S[i]],[theta_1,theta1[j]],[theta_2,theta2[j]],[theta_3,theta3[j]]]))
    

In [174]:
V1st = np.zeros((len(S),len(t),2,1));
V2st = np.zeros((len(S),len(t),2,1));
V3st = np.zeros((len(S),len(t),2,1));

#  calculating velocity values
for i in range(len(S)):
    for j in range(len(t)):
        dTHETA =  np.array([[dtheta1[j]],[dtheta2[j]],[dtheta3[j]]])
        V1st[i][j] = J1st[i,j] @ dTHETA
        V2st[i][j] = J2st[i,j] @ dTHETA 
        V3st[i][j] = J3st[i,j] @ dTHETA 

F1st = -V1st
F2st = -V2st
F3st = -V3st

In [175]:
plt.close();
fig1 = plt.figure(figsize=(10,10))

In [190]:
sc = 50
for i in range(len(t)):
    plt.clf();
    plt.axis([-3,3,-3,3])
    ax = plt.gca()
    ax.plot([[0],FK1st[10,i,0],FK2st[10,i,0],FK3st[10,i,0]],[[0],FK1st[10,i,1],FK2st[10,i,1],FK3st[10,i,1]],'g')
    ax.plot(FK1st[0,i,0],FK1st[0,i,1],'.k')
    ax.plot(FK2st[0,i,0],FK2st[0,i,1],'.k')
    ax.plot(FK3st[0,i,0],FK3st[0,i,1],'.k')
    ax.quiver(FK1st[:,i,0],FK1st[:,i,1],F1st[:,i,0],F1st[:,i,1],scale=sc)
    ax.quiver(FK2st[:,i,0],FK2st[:,i,1],F2st[:,i,0],F2st[:,i,1],scale=sc)
    ax.quiver(FK3st[:,i,0],FK3st[:,i,1],F3st[:,i,0],F3st[:,i,1],scale=sc)
    
    ax.plot([[0],-FK1st[10,i,0],-FK2st[10,i,0],-FK3st[10,i,0]],[[0],FK1st[10,i,1],FK2st[10,i,1],FK3st[10,i,1]],'g')
    ax.plot(-FK1st[0,i,0],FK1st[0,i,1],'.k')
    ax.plot(-FK2st[0,i,0],FK2st[0,i,1],'.k')
    ax.plot(-FK3st[0,i,0],FK3st[0,i,1],'.k')
    ax.quiver(-FK1st[:,i,0],FK1st[:,i,1],-F1st[:,i,0],F1st[:,i,1],scale=sc)
    ax.quiver(-FK2st[:,i,0],FK2st[:,i,1],-F2st[:,i,0],F2st[:,i,1],scale=sc)
    ax.quiver(-FK3st[:,i,0],FK3st[:,i,1],-F3st[:,i,0],F3st[:,i,1],scale=sc)

    plt.show()
    plt.draw()
    plt.pause(0.05)

In [152]:
fig2 = plt.figure(figsize=(10,10))

In [236]:
Fxy1 = np.mean(F1st,axis=0)
Fxy2 = np.mean(F2st,axis=0)
Fxy3 = np.mean(F3st,axis=0)

Fxy = np.mean(F1st,axis=0) + np.mean(F2st,axis=0) + np.mean(F3st,axis=0)
Fav = np.mean(Fxy,axis=0)
plt.plot(Fxy1[:,0],Fxy1[:,1])
plt.quiver(0,0,Fav[0],Fav[1],scale=10)

In [167]:
plt.plot(t,Fxy[:,0])
plt.plot(t,Fxy[:,1])

In [218]:
tau = np.zeros((3,1,len(t)))
for i  in range(len(t)):
    tau[:,:,i] = J3st[10,i].transpose() @ Fxy[i]

In [224]:
plt.plot(t,tau[0,0,:])
plt.plot(t,tau[1,0,:])
plt.plot(t,tau[2,0,:])

In [207]:
np.shape(J1st.transpose())

(3, 2, 100, 11)